# Game Play Student Performance 1
### First notebook
### Get baseline Brier Score

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Reference: https://www.kaggle.com/competitions/predict-student-performance-from-game-play/discussion/384359
dtypes={
    'elapsed_time':np.int32,
    'name':'category',
    'level':np.uint8,
    'room_coor_x':np.float32,
    'room_coor_y':np.float32,
    'screen_coor_x':np.float32,
    'screen_coor_y':np.float32,
    'hover_duration':np.float32,
    'fullscreen':np.uint8,
    'hq':np.uint8,
    'music':np.uint8,
    'level_group':'category'}

In [3]:
raw_data_train = pd.read_csv("./student_perf_data_raw/train.csv", dtype=dtypes)
print(len(raw_data_train))
raw_data_test = pd.read_csv("./student_perf_data_raw/test.csv", dtype=dtypes)
print(len(raw_data_test))


26296946
3728


In [4]:
print(len(raw_data_test)+len(raw_data_train))

26300674


In [5]:
#combine train and test set for feature engineering use later
raw_data_train['label'] = 'train'
raw_data_test['label'] = 'test'
combined_set = pd.concat([raw_data_train, raw_data_test], axis=0).reset_index(drop=True)
print(len(combined_set))
combined_set.tail()

26300674


,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,label,session_level
26300669,20090312331414616,1001,1581679,map_hover,basic,22,NaN,NaN,NaN,NaN,...,NaN,tunic.wildlife,tunic.historicalsociety.entry,NaN,0,0,1,13-22,test,8.0
26300670,20090312331414616,1002,1583044,map_hover,basic,22,NaN,NaN,NaN,NaN,...,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,0,0,1,13-22,test,8.0
26300671,20090312331414616,1003,1583410,map_click,undefined,22,NaN,483.726349,-3.880048,456.0,...,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,0,0,1,13-22,test,8.0
26300672,20090312331414616,1004,1585841,navigate_click,undefined,22,NaN,192.372147,38.216179,383.0,...,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,0,0,1,13-22,test,8.0
26300673,20090312331414616,1005,1587810,checkpoint,basic,22,NaN,NaN,NaN,NaN,...,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,0,0,1,13-22,test,8.0


# Get random set to speed up process

In [6]:
unique_sessions = combined_set.loc[:, ['session_id']].drop_duplicates()
random_set = unique_sessions.sample(frac=0.1, replace=False).reset_index(drop=True)

In [7]:
# Set combined set to random set to reduce size
combined_set = random_set.merge(combined_set.rename_axis('temp_index').reset_index(), how='left', on='session_id').set_index('temp_index', drop=True)
combined_set.index.names = [""]

In [8]:
combined_set.tail()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,label,session_level
,,,,,,,,,,,,,,,,,,,,,
2463279,20110311060547370,1362,1898290,map_hover,basic,22,NaN,NaN,NaN,NaN,...,NaN,tomap,tunic.historicalsociety.entry,NaN,0,0,1,13-22,train,NaN
2463280,20110311060547370,1363,1898739,map_hover,basic,22,NaN,NaN,NaN,NaN,...,NaN,tunic.humanecology,tunic.historicalsociety.entry,NaN,0,0,1,13-22,train,NaN
2463281,20110311060547370,1364,1899089,map_click,undefined,22,NaN,532.578979,49.819336,489.0,...,NaN,tunic.capitol_2,tunic.historicalsociety.entry,NaN,0,0,1,13-22,train,NaN
2463282,20110311060547370,1365,1903054,navigate_click,undefined,22,NaN,262.069122,104.882843,429.0,...,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,0,0,1,13-22,train,NaN
2463283,20110311060547370,1366,1905030,checkpoint,basic,22,NaN,NaN,NaN,NaN,...,NaN,chap4_finale_c,tunic.capitol_2.hall,NaN,0,0,1,13-22,train,NaN


In [9]:
len(combined_set)

2630509

# Exploratory Analysis

In [10]:
explore_set = combined_set[combined_set['label']=='train']

In [11]:
explore_set.info(verbose=True, show_counts=True)
print(len(explore_set))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2630509 entries, 5890842 to 2463283
Data columns (total 22 columns):
 #   Column          Non-Null Count    Dtype   
---  ------          --------------    -----   
 0   session_id      2630509 non-null  int64   
 1   index           2630509 non-null  int64   
 2   elapsed_time    2630509 non-null  int32   
 3   event_name      2630509 non-null  object  
 4   name            2630509 non-null  category
 5   level           2630509 non-null  uint8   
 6   page            56747 non-null    float64 
 7   room_coor_x     2423043 non-null  float32 
 8   room_coor_y     2423043 non-null  float32 
 9   screen_coor_x   2423043 non-null  float32 
 10  screen_coor_y   2423043 non-null  float32 
 11  hover_duration  200365 non-null   float32 
 12  text            961827 non-null   object  
 13  fqid            1804360 non-null  object  
 14  room_fqid       2630509 non-null  object  
 15  text_fqid       961833 non-null   object  
 16  fullscreen  

In [12]:
# Consider to split event name into event and action
# When both room coordinates are positive, room is in sight. x
### Might be useful to know how many steps to room in sight for each level / event
### Might also be useful to know how many steps stayed when room is in sight
# Good to know time used at each step (index) x
# Good to know how many of each even, like how many person talked to (clicked), how many maps seen etc. x
# Good to know how many times the same map / person is clicked. (repeated actions) x
# Good to know if someone uses music / fullscreen or change halfway (use avg) x
# Good to know if text is a question 
# Good to know page access count for each session / level / level grp
# Good to know lead lag event_name
# Good to know text info after extracting room/fqid info

### Prelim Feature Engineering prior to aggregation

In [13]:
print(len(combined_set.index))
print(combined_set.index.nunique())

2630509
2630509


In [14]:
# convert dummy
# Split event_name into event and action
print(len(combined_set))
combined_set["event"] = combined_set["event_name"].apply(lambda x: x.split("_")[0])
combined_set["action"] = combined_set["event_name"].apply(lambda x: x.split("_")[-1])

2630509


In [15]:
# convert dummy
# Get event type count for session, level and level grp 
#event
print(len(combined_set))
combined_set['level_event_count'] = combined_set.groupby(['session_id', 'level', 'event'])['event'].transform('count')
#action
print(len(combined_set))
combined_set['level_action_count'] = combined_set.groupby(['session_id', 'level', 'action'])['action'].transform('count')

2630509
2630509


In [17]:
# convert dummy
# Get count of repeated actions
#repeated actions

def q75(x):
    return x.quantile(0.75)

print(len(combined_set))
combined_set['rptevent'] = combined_set['event_name'] +['_']+ combined_set['fqid']
combined_set['level_rptevent_count'] = combined_set.groupby(['session_id', 'level', 'rptevent'])['rptevent'].transform('count')
combined_set['level_rptevent_quantile'] = combined_set.groupby(['session_id','level'])['level_rptevent_count'].transform(q75)
level_rptevent_sigcount = combined_set['level_rptevent_count'][combined_set['level_rptevent_count'].gt(combined_set['level_rptevent_quantile'])]
level_rptevent_sigcount = pd.DataFrame(level_rptevent_sigcount).set_axis(["level_rptevent_sigcount"], axis='columns')
combined_set = pd.concat([combined_set, level_rptevent_sigcount], axis=1)
combined_set.drop(['level_rptevent_count', 'level_rptevent_quantile'], axis=1, inplace=True)

2630509


In [19]:
# convert dummy
# Get count of room id
# access room how many times per level
print(len(combined_set))
combined_set['level_roomfqid_count'] = combined_set.groupby(['session_id', 'level', 'room_fqid'])['room_fqid'].transform('count')

2630509


In [20]:
# Check if room in sight, and general positional info
print(len(combined_set))
combined_set['room_in_sight'] = np.where((combined_set['room_coor_x']>0) & (combined_set['room_coor_y']>0), True, False)
print(len(combined_set))
combined_set['room_position_info'] =  abs(combined_set['screen_coor_x']-combined_set['room_coor_x'])+abs(combined_set['screen_coor_y']-combined_set['room_coor_y'])
print(len(combined_set))
combined_set['room_not_in_sight_count'] = combined_set[~combined_set['room_in_sight']].groupby(['session_id', 'level'])['room_in_sight'].transform('count')
combined_set['room_not_in_sight_count'] = combined_set.groupby(['session_id', 'level'])['room_not_in_sight_count'].ffill().bfill()

2630509
2630509
2630509


In [21]:
# Get steps for level and level group
combined_set['level_step_count'] = combined_set.groupby(['session_id', 'level']).cumcount()

In [22]:
# Get time use at each step for level and level group
combined_set['level_step_time_used'] = combined_set.groupby(['session_id', 'level'])['elapsed_time'].diff().fillna(0)
combined_set['level_elapsed_time'] = combined_set.groupby(['session_id', 'level'])['level_step_time_used'].cumsum()

In [23]:
# Check music and fullscreen used for full session
combined_set['full_sess_music'] = combined_set.groupby(['session_id'])['music'].transform('mean')
combined_set['full_sess_fullscreen'] = combined_set.groupby(['session_id'])['fullscreen'].transform('mean')
combined_set['full_sess_hq'] = combined_set.groupby(['session_id'])['hq'].transform('mean')

In [24]:
# Check if text is a question
combined_set['text_is_question'] = np.where((~combined_set['text'].isnull()) & (combined_set['text'].str.contains('\?')), True, False)
combined_set['text_is_question_count'] = combined_set[combined_set['text_is_question']].groupby(['session_id', 'level'])['text_is_question'].transform('count')
combined_set['text_is_question_count'] = combined_set.groupby(['session_id', 'level'])['text_is_question_count'].ffill().bfill()

In [25]:
# convert dummy
# Get page access count
combined_set['level_page_access_count'] = combined_set.groupby(['session_id', 'level', 'page'])['page'].transform('count')

In [26]:
# Get additional text info if available
fqid = combined_set['fqid'].str.lower()
text_fqid = combined_set['text_fqid'].str.lower()
room_fqid = combined_set['room_fqid'].str.lower()      

In [27]:
df_temp = pd.DataFrame({'fqid':fqid, 'text_fqid':text_fqid})
df_checked = df_temp.apply(lambda x: [word for word in str(x.text_fqid).split(".") if word not in str(x.fqid).split('.')] , axis=1) 

In [28]:
df_temp = pd.DataFrame({'room_fqid':room_fqid, 'df_checked':df_checked})
df_temp['df_checked'] = ['.'.join(map(str,l)) for l in df_temp['df_checked']]

In [29]:
df_checked = df_temp.apply(lambda x: [word for word in str(x.df_checked).split(".") if word not in str(x.room_fqid).split('.')] , axis=1) 
df_checked = ['.'.join(map(str,l)) for l in df_checked]

In [30]:
combined_set['addition_info_fr_id'] = df_checked
combined_set['addition_info_fr_id'].replace(['', 'nan'], np.nan, inplace=True)
combined_set.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,level_step_count,level_step_time_used,level_elapsed_time,full_sess_music,full_sess_fullscreen,full_sess_hq,text_is_question,text_is_question_count,level_page_access_count,addition_info_fr_id
,,,,,,,,,,,,,,,,,,,,,
5890842,21010108291765796,0,0,cutscene_click,basic,0,NaN,-402.991394,26.685314,391.0,...,0,0.0,0.0,1.0,0.0,0.0,False,3.0,NaN,NaN
5890843,21010108291765796,1,25472936,person_click,basic,0,NaN,-183.991409,-0.314686,610.0,...,1,25472936.0,25472936.0,1.0,0.0,0.0,True,3.0,NaN,intro_0_cs_0
5890844,21010108291765796,2,25476068,person_click,basic,0,NaN,33.008595,-150.314682,827.0,...,2,3132.0,25476068.0,1.0,0.0,0.0,False,3.0,NaN,intro_0_cs_0
5890845,21010108291765796,3,25478455,person_click,basic,0,NaN,33.008595,-151.314682,827.0,...,3,2387.0,25478455.0,1.0,0.0,0.0,False,3.0,NaN,intro_0_cs_0
5890846,21010108291765796,4,25480312,person_click,basic,0,NaN,33.008595,-151.314682,827.0,...,4,1857.0,25480312.0,1.0,0.0,0.0,False,3.0,NaN,intro_0_cs_0


In [31]:
# convert dummy
# Get count of room id
# access room how many times per level

def q75(x):
    return x.quantile(0.75)

combined_set['level_additionalinfo_count'] = combined_set.groupby(['session_id', 'level', 'addition_info_fr_id'])['addition_info_fr_id'].transform('count')
combined_set['level_additionalinfo_quantile'] = combined_set.groupby(['session_id','level'])['level_additionalinfo_count'].transform(q75)
level_additionalinfo_sigcount = combined_set['level_additionalinfo_count'][combined_set['level_additionalinfo_count'].gt(combined_set['level_additionalinfo_quantile'])]
level_additionalinfo_sigcount = pd.DataFrame(level_additionalinfo_sigcount).set_axis(["level_additionalinfo_sigcount"], axis='columns')
combined_set = pd.concat([combined_set, level_additionalinfo_sigcount], axis=1)
combined_set.drop(['level_additionalinfo_count', 'level_additionalinfo_quantile'], axis=1, inplace=True)

In [33]:
combined_set.to_csv('./student_perf1/after_interim_feature_engineering.csv')

### Preprocessing for aggregation

In [34]:
reduced_combined_set = combined_set.drop(['index', 'event_name', 'name', 
                                          'room_coor_x', 'room_coor_y', 
                                          'screen_coor_x', 'screen_coor_y', 'text',
                                         'fqid', 'fullscreen', 'hq', 'music',
                                         'text_fqid', 'room_in_sight', 'text_is_question',
                                         'level_step_time_used'], axis=1)

In [35]:
# reduced_combined_set.to_csv('./student_perf1/reduced_df_before_aggregation.csv')

In [36]:
# Pivot categorical columns

In [37]:
reduced_combined_set = pd.read_csv('./student_perf1/after_interim_feature_engineering.csv')

In [38]:
reduced_combined_set.shape

(2630509, 44)

In [39]:
# pivot and updown fill room_fqid
temp = reduced_combined_set.pivot(columns='room_fqid', values='level_roomfqid_count')
temp2 = pd.concat([reduced_combined_set[['session_id', 'level']], temp], axis=1)
temp2 = temp2.groupby(['session_id', 'level'])[['session_id', 'level'] + list(pd.Series(list(temp.columns)).explode())].ffill().bfill()
temp2.drop(['session_id', 'level'], axis=1, inplace=True)
reduced_combined_set = pd.concat([reduced_combined_set, temp2], axis=1).drop(['room_fqid','level_roomfqid_count'], axis=1)
reduced_combined_set.shape

(2630509, 61)

In [40]:
# pivot event
temp = reduced_combined_set.pivot(columns='event', values='level_event_count')
temp2 = pd.concat([reduced_combined_set[['session_id', 'level']], temp], axis=1)
temp2 = temp2.groupby(['session_id', 'level'])[['session_id', 'level'] + list(pd.Series(list(temp.columns)).explode())].ffill().bfill()
temp2.drop(['session_id', 'level'], axis=1, inplace=True)
reduced_combined_set = pd.concat([reduced_combined_set, temp2], axis=1).drop(['event','level_event_count'], axis=1)
reduced_combined_set.shape

(2630509, 68)

In [41]:
# pivot action
temp = reduced_combined_set.pivot(columns='action', values='level_action_count')
temp2 = pd.concat([reduced_combined_set[['session_id', 'level']], temp], axis=1)
temp2 = temp2.groupby(['session_id', 'level'])[['session_id', 'level'] + list(pd.Series(list(temp.columns)).explode())].ffill().bfill()
temp2.drop(['session_id', 'level'], axis=1, inplace=True)
reduced_combined_set = pd.concat([reduced_combined_set, temp2], axis=1).drop(['action','level_action_count'], axis=1)
reduced_combined_set.shape

(2630509, 69)

In [42]:
#pivot rptevent
temp = reduced_combined_set.pivot(columns='rptevent', values='level_rptevent_sigcount')
temp2 = pd.concat([reduced_combined_set[['session_id', 'level']], temp], axis=1)
temp2 = temp2.groupby(['session_id', 'level'])[['session_id', 'level'] + list(pd.Series(list(temp.columns)).explode())].ffill().bfill()
temp2.drop(['session_id', 'level'], axis=1, inplace=True)
reduced_combined_set = pd.concat([reduced_combined_set, temp2], axis=1).drop(['rptevent','level_rptevent_sigcount'], axis=1)
reduced_combined_set.shape

(2630509, 344)

In [43]:
#pivot page
temp = reduced_combined_set.pivot(columns='page', values='level_page_access_count')
temp2 = pd.concat([reduced_combined_set[['session_id', 'level']], temp], axis=1)
temp2 = temp2.groupby(['session_id', 'level'])[['session_id', 'level'] + list(pd.Series(list(temp.columns)).explode())].ffill().bfill()
temp2.drop(['session_id', 'level'], axis=1, inplace=True)
reduced_combined_set = pd.concat([reduced_combined_set, temp2], axis=1).drop(['page','level_page_access_count'], axis=1)
reduced_combined_set.shape

(2630509, 350)

In [44]:
#pivot additional info
temp = reduced_combined_set.pivot(columns='addition_info_fr_id', values='level_additionalinfo_sigcount')
temp2 = pd.concat([reduced_combined_set[['session_id', 'level']], temp], axis=1)
temp2 = temp2.groupby(['session_id', 'level'])[['session_id', 'level'] + list(pd.Series(list(temp.columns)).explode())].ffill().bfill()
temp2.drop(['session_id', 'level'], axis=1, inplace=True)
reduced_combined_set = pd.concat([reduced_combined_set, temp2], axis=1).drop(['addition_info_fr_id','level_additionalinfo_sigcount'], axis=1)
reduced_combined_set.shape

(2630509, 418)

In [61]:
reduced_combined_set.drop('Unnamed: 0', axis=1, inplace=True)
reduced_combined_set.info(verbose=True)

MemoryError: Unable to allocate 6.19 GiB for an array with shape (316, 2630509) and data type float64

In [46]:
# aggregate to session and level

In [47]:
agg_combined_set = reduced_combined_set.groupby(['session_id', 'level']).agg({
    'elapsed_time': 'max',
    'hover_duration': 'mean',
    'room_position_info': 'mean',
    'room_not_in_sight_count': 'sum',
    'level_step_count': 'max',
    'level_elapsed_time': 'max',
    'full_sess_music': 'mean',
    'full_sess_hq': 'mean',
    'full_sess_fullscreen': 'mean',
    'text_is_question_count': 'sum',
    'level_group': 'max',
    'label': 'max'
})
agg_combined_set.tail()

elapsed_time  hover_duration  room_position_info  \
session_id        level                                                     
22100211280762644 18          1116267      887.000000          996.483957   
                  19          1157483      142.000000         1098.681394   
                  20          1186960     2609.000000          868.338647   
                  21          1250189     1339.363636          813.543397   
                  22          1263910      266.000000          734.877068   

                         room_not_in_sight_count  level_step_count  \
session_id        level                                              
22100211280762644 18                      7979.0               100   
                  19                       663.0                38   
                  20                       475.0                24   
                  21                      3036.0                68   
                  22                        40.0                 7   

                         level_elapsed_time  full_sess_music  full_sess_hq  \
session_id        level                                                      
22100211280762644 18               116138.0              1.0           0.0   
                  19                38743.0              1.0           0.0   
                  20                29013.0              1.0           0.0   
                  21                62716.0              1.0           0.0   
                  22                12106.0              1.0           0.0   

                         full_sess_fullscreen  text_is_question_count  \
session_id        level                                                 
22100211280762644 18                      0.0                  1111.0   
                  19                      0.0                   195.0   
                  20                      0.0                    25.0   
                  21                      0.0                   207.0   
                  22                      0.0                    24.0   

                        level_group  label  
session_id        level                     
22100211280762644 18          13-22  train  
                  19          13-22  train  
                  20          13-22  train  
                  21          13-22  train  
                  22          13-22  train

In [48]:
agg_combined_set.to_csv('./student_perf1/agg_dataset.csv')

In [49]:
#merge pivot set
print(len(reduced_combined_set))
pivoted_set = reduced_combined_set.set_index(['session_id', 'level']).drop(agg_combined_set.columns, axis=1).drop_duplicates()
print(len(pivoted_set))

2630509
2630509


In [50]:
# pivoted_set.to_csv('./student_perf1/check_pivotedset.csv')

In [51]:
pivoted_set.tail()

Unnamed: 0  index      event_name       name  \
session_id        level                                                 
20110311060547370 22        2463279   1362       map_hover      basic   
                  22        2463280   1363       map_hover      basic   
                  22        2463281   1364       map_click  undefined   
                  22        2463282   1365  navigate_click  undefined   
                  22        2463283   1366      checkpoint      basic   

                         room_coor_x  room_coor_y  screen_coor_x  \
session_id        level                                            
20110311060547370 22             NaN          NaN            NaN   
                  22             NaN          NaN            NaN   
                  22       532.57900    49.819336          489.0   
                  22       262.06912   104.882840          429.0   
                  22             NaN          NaN            NaN   

                         screen_coor_y text                fqid  ...  \
session_id        level                                          ...   
20110311060547370 22               NaN  NaN               tomap  ...   
                  22               NaN  NaN  tunic.humanecology  ...   
                  22             299.0  NaN     tunic.capitol_2  ...   
                  22             228.0  NaN      chap4_finale_c  ...   
                  22               NaN  NaN      chap4_finale_c  ...   

                        symbol_recap  takealook  talktogramps  \
session_id        level                                         
20110311060547370 22             NaN        NaN           NaN   
                  22             NaN        NaN           NaN   
                  22             NaN        NaN           NaN   
                  22             NaN        NaN           NaN   
                  22             NaN        NaN           NaN   

                         tracks.hub.deer  trapped  wells  wells_recap  \
session_id        level                                                 
20110311060547370 22                 NaN      NaN    NaN          NaN   
                  22                 NaN      NaN    NaN          NaN   
                  22                 NaN      NaN    NaN          NaN   
                  22                 NaN      NaN    NaN          NaN   
                  22                 NaN      NaN    NaN          NaN   

                         wellsbadge.hub  whatdo  writeitup  
session_id        level                                     
20110311060547370 22                NaN     NaN        NaN  
                  22                NaN     NaN        NaN  
                  22                NaN     NaN        NaN  
                  22                NaN     NaN        NaN  
                  22                NaN     NaN        NaN  

[5 rows x 404 columns]

In [57]:
full_agg_set = pd.concat([agg_combined_set, pivoted_set], axis=1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [60]:
import gc
gc.collect()

2041

# Explore Aggr Set

In [ ]:
full_agg_set.tail()

In [ ]:
full_agg_set.info(verbose=True, show_counts=True)

In [ ]:
full_agg_set.to_csv('./student_perf1/full_agg_set.csv')